### Categorization of Customer Service Inquiries ###

We will retrieve the allowable categories from two .csvs (categories_primary.csv and categories_secondary.csv).  The data source could also be a SQL db or other call to a system database.  We will use pandas to read the files.

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [2]:
import pandas as pd

Import opanai and dotenv libraries.
Load API key into environment from .env file which contains the line "open_ai_key = [insert your API key here]"
Instatiate client using OpenAI() functiorary

In [19]:
import openai
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

client = openai.OpenAI()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Define a function called get_output_from_messages.
When called, this function will pass the prompt to OpenAI, along wtih other parameters (many optional) that we set. The function will return the completion generated by the call to the OpenAI LLM.

In [5]:
def get_output_from_messages(messages_input, model_input = "gpt-3.5-turbo", temperature_input = 0):
    response = client.chat.completions.create(
        model = model_input,
        messages = messages_input,
        temperature = temperature_input,
    )

    response = response.choices[0].message.content
    return response

Read the .csv file using pandas read_csv() function.  

In [7]:
primary_df = pd.read_csv('drive/MyDrive/Colab Notebooks/GBA 478/Week 3/Categorization/categories_primary.csv')
primary = primary_df["Primary Categories"].tolist()
primary

['Harware and Network',
 'Subscriptions and Billing',
 'Games and apps',
 'Account and Profile',
 'Online Safety',
 'Social Activity']

In [8]:
secondary_df = pd.read_csv('drive/MyDrive/Colab Notebooks/GBA 478/Week 3/Categorization/categories_secondary.csv')
secondary = secondary_df.groupby('Primary Category')['Secondary Category'].apply(list).to_dict()
secondary

{'Account and Profile': ['New account creation',
  'Forgot username',
  'Forgt password',
  'Change profile information',
  'Delete account',
  'Two factor authentification'],
 'Games and apps': ['Game not working',
  'App not working',
  'Game backward compatability',
  'Game updates',
  'Downloadable content'],
 'Harware and Network': ['Console Setup',
  'Network setup',
  'Console troubleshooting',
  'Video issue',
  'Audio issue',
  'Controller not working',
  'Accessory not working',
  'System updates'],
 'Online Safety': ['Privacy settings',
  'Report onine harassment',
  'Report another user',
  'Account ban or suspension'],
 'Social Activity': ['Help with friends and followers ',
  'Voice chat not working',
  "Can't send or receive messages",
  'Game capture issues',
  'Multiplayer hosting'],
 'Subscriptions and Billing': ['Preordered content',
  'Purchased content',
  'Gifted content',
  'Subscriptions',
  'Refunds',
  'Charge disputes',
  'Recurring billing',
  'Balance inqui

In [9]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries delimited with {delimiter} characters.
Classify each query into a primary category and a secondary category.
"""

In [16]:
user_message = f"There is no sound whenI play Fortnite"
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]
response = get_output_from_messages(messages)
print(response)



{
    "primary": "Harware and Network",
    "secondary": "Audio issue"
}


In [11]:
type(response)

str

In [12]:
response['primary']

TypeError: string indices must be integers

Consider how we might alter the system message to give more structure to the output.

In [15]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries delimited with {delimiter} characters.
Classify each query into a primary category and a secondary category.
Provide your output in json format with the keys: primary and secondary.

Primary categories: {primary}

Secondary categories: {secondary}

"""

Import the library 'json' to parse string in valid JSON format and return a Python dictionary.

JSON ("Javascript Object Notation") is a data format used in many languages.  JSON objects are string (plain text) with a defined structure and notation.  JSON are commonly used to store and communicate data between programs or applications.

In [17]:
import json

response = json.loads(response)
response

{'primary': 'Harware and Network', 'secondary': 'Audio issue'}

In [18]:
response['primary']

'Harware and Network'